# 📘 Easing Functions & Perlin Noise
Easing functions control how values transition over time, making animations and data curves **smooth and natural**. Perlin noise adds **realistic variations**, commonly used in **graphics, simulations, and time-series generation**.

### 🔹 What Are Easing Functions?
Instead of moving at a constant speed, easing functions create **acceleration and deceleration effects**. Examples:
- **CubicEaseInOut** → Smooth start & end  
- **QuadEaseIn** → Slow start, fast finish  
- **CircularEaseIn** → Simulates circular motion  

### 🔹 Why Use Perlin Noise?
Perlin noise introduces **fluid randomness**, making transitions **less artificial**. It’s great for **terrain generation, animations, and synthetic data**.

### 📌 What This Notebook Covers
✅ Visualizing easing functions (**Matplotlib**)  
✅ Adding Perlin noise for realistic variation  
✅ Generating synthetic data with **`easeflow`** (PySpark)  

👉 **Run the next cells to explore these concepts interactively!** 🚀  


# Introduction to Easing Functions & Perlin Noise (Matplotlib)

 > Skip this section if you are familiar with both Easing functions and Perlin noise. 

Before diving in the simplified approach the `easeflow` provided, let's check a simple example using Matplot lib and numpy.

In the next cell, we will visualize various easing functions and see how Perlin noise can be applied to them. This will help us understand the impact of noise on smooth transitions and how it can make animations and synthetic data more dynamic and realistic.

We will use interactive widgets to adjust the noise parameters and observe the changes in real-time. This interactive approach allows for a deeper exploration of how different easing functions behave under the influence of Perlin noise.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, FloatSlider
from perlin_noise import PerlinNoise
import easing_functions as easy
from pyspark.sql.functions import lit

# Global PerlinNoise object
noise = PerlinNoise(octaves=10, seed=1)


def apply_noise(easing: callable, x: np.ndarray, noise_max: float, noise_speed: float) -> np.ndarray:
    """
    Applies Perlin noise to a given easing function to create more natural variation.

    Easing functions produce smooth curves that can represent animations, synthetic data,
    or interpolation. By adding Perlin noise, we introduce **controlled randomness** 
    while keeping the general shape of the easing function.

    Args:
        easing (callable): The easing function (e.g., CubicEaseInOut).
        x (np.ndarray): A normalized array of values from 0 to 1.
        noise_max (float): The maximum noise influence (0 = no noise).
        noise_speed (float): Controls noise frequency (higher = more variation).

    Returns:
        np.ndarray: The noisy easing function values.
    """
    noise_func = np.vectorize(lambda val: val * (1 - noise_max + noise_max * (1 + noise(val * noise_speed))))
    return noise_func(easing(x))


def plot_subplot(ax: plt.Axes, easing_function: callable, x: np.ndarray, noise_max: float, noise_speed: float) -> None:
    """
    Plots an easing function with and without Perlin noise.

    This helps visualize how Perlin noise modifies smooth transitions, making 
    easing functions more dynamic and realistic.

    Args:
        ax (plt.Axes): The subplot axis for visualization.
        easing_function (callable): The easing function (e.g., QuinticEaseIn).
        x (np.ndarray): A normalized array of values from 0 to 1.
        noise_max (float): The maximum noise influence.
        noise_speed (float): Frequency of the noise variation.
    """
    easing = np.vectorize(easing_function(start=0.0, end=1.0))
    
    # Compute smooth and noisy curves
    y_smooth = easing(x)
    y_noisy = apply_noise(easing, x, noise_max, noise_speed)

    ax.set_title(easing_function.__name__, fontsize=10)
    ax.plot(x, y_smooth, label="No Noise", linewidth=2)
    ax.plot(x, y_noisy, label="With Perlin Noise", linestyle="solid", alpha=0.8)
    ax.legend()
    ax.grid(True)


def plot_examples(noise_max: float = 0.3, noise_speed: float = 0.001) -> None:
    """
    Plots multiple easing functions with and without Perlin noise.

    This example is useful for **understanding** how easing functions work 
    and how Perlin noise modifies them.

    Args:
        noise_max (float): Maximum noise influence (default: 0.3).
        noise_speed (float): Frequency of noise variation (default: 0.001).
    """
    functions = [easy.CubicEaseInOut, easy.QuadEaseIn, easy.QuinticEaseIn, easy.CircularEaseIn]
    
    num_functions = len(functions)
    num_cols = 2  # Fixed number of columns for better visualization
    num_rows = math.ceil(num_functions / num_cols)

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 6), sharex=True, sharey=True)
    fig.suptitle("Examples of Easing Functions with Noise Applied", fontsize=12)

    x_values = np.linspace(0, 1, 1000)  # High-resolution curve

    for i, easing_function in enumerate(functions):
        row, col = divmod(i, num_cols)
        plot_subplot(axs[row, col], easing_function, x_values, noise_max, noise_speed)

    # Remove empty subplots if an odd number of functions
    if num_functions % num_cols != 0:
        fig.delaxes(axs[-1, -1])

    plt.tight_layout()
    plt.show()


def show_interactive():
    """
    Interactive widget for adjusting noise parameters and visualizing easing functions.

    This allows users to explore how Perlin noise affects different easing functions.
    """
    noise_max_slider = FloatSlider(value=0.3, min=0.0, max=5.0, step=0.1, description="Noise Max")
    noise_speed_slider = FloatSlider(value=2, min=0.0, max=5.0, step=0.01, description="Noise Speed")
    
    interact(plot_examples, noise_max=noise_max_slider, noise_speed=noise_speed_slider)

show_interactive()


interactive(children=(FloatSlider(value=0.3, description='Noise Max', max=5.0), FloatSlider(value=2.0, descrip…

# Part 2 - Using `easeflow` for Synthetic Data Generation (Spark)

`easeflow` provides two key functions for synthetic data generation:

- **`norm_df(n: int)`** → Returns a DataFrame with:
  - `id` → Sequential index (0 to `n-1`)
  - `t` → Normalized values (0 to 1)

- **`make_udf(easing_function, min_val, max_val)`** → Creates a **PySpark UDF** that applies an easing function, optionally modified with Perlin noise.

### 🔹 Generating a Normalized DataFrame
To create a structured dataset for easing functions:
```python
df = norm_df(365)
display(df)
```

Output:  
```
+---+------------------+  
| id|                 t|  
+---+------------------+  
|  0|  0.0             |  
|  1|  0.00274         |  
|  2|  0.00548         |  
|...|  ...             |  
|364|  1.0             |  
+---+------------------+  
```


## 🔹 Applying an Easing Function with Noise  

We can now use make_udf to apply an easing function to this dataset.

In [1]:
import sys

sys.path.append('../')


In [3]:
from easeflow import make_udf, norm_df, ease
from pyspark.sql import functions as F

# Create an easing function UDF
easing_udf = make_udf(ease.QuinticEaseIn, min_val=500, max_val=1000)

# Apply the easing function with and without noise
df = (
    norm_df(365)
    .withColumn("v", easing_udf(F.col("t"), F.lit(0)))  # No noise
    .withColumn("v_noise", easing_udf(F.col("t"), F.lit(0.3)))  # With Perlin noise
)

display(df)

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/serializers.py", line 192, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/serializers.py", line 609, in loads
    return cloudpickle.loads(obj, encoding=encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'easing_functions'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/worker.py", line 2029, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/databricks/spark/python/pyspark/worker.py", line 1916, in read_udfs
    read_single_udf(
  File "/databricks/spark/python

DataFrame[id: bigint, t: double, v: float, v_noise: float]

## 🔹 Adding Time-Series Data

Since norm_df() provides an id column, we can map it to a date column to simulate a time-series dataset.

In [ ]:
from datetime import datetime, timedelta

# Define start date
dataset_len = 365
start_date = datetime.today().replace(day=1) - timedelta(days=dataset_len)

# Generate dataset with a date column
df = (
    norm_df(dataset_len)
    .withColumn("date", F.date_add(F.lit(start_date), F.col("id").cast("integer")))  # Convert `id` to a date
    .withColumn("v", easing_udf(F.col("t"), F.lit(0)))  # No noise
    .withColumn("v_noise", easing_udf(F.col("t"), F.lit(0.3)))  # With noise
)

display(df)
